In [123]:
%matplotlib widget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import json
from IPython.display import Markdown, display

def plot_frames(ft_err, speed):
    fig, ax1 = plt.subplots()

    color = 'tab:blue'
    ax1.set_xlabel('sample')
    ax1.set_ylabel('frametime', color=color)
    ax1.plot(ft_err, color=color)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:red'
    ax2.set_ylabel('speed', color=color)  # we already handled the x-label with ax1
    ax2.plot(speed, color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()

def analyse_frames_from_dataframe(data):
    data = data[2:]
    x = data[0]
    ft = data[1]
    speed = data[3]

    ft_err = abs(ft - 1000/60)
    duration = x.iloc[-1] - x.iloc[0]
    ft_err_sum = ft_err.sum()
    ft_err_norm_sum = ft_err_sum / duration
    #print(f"Frametime sum of absolute errors: {round(ft_err_sum, 2)} ms")
    #print(f"Frametime lag time per frame: {round(ft_err_norm_sum * 100)}%")
    
    ft_err_speed = pd.DataFrame({
        "error": ft_err,
        "speed": speed,
        "ft": ft,
    })
    ft_err_speed_sum = ft_err_speed.groupby("speed").sum()
    ft_err_speed_norm_sum = ft_err_speed_sum["error"] / ft_err_speed_sum["ft"]
    
    return ft_err_norm_sum, ft_err_speed_norm_sum
    
    #display(ft_err_speed_norm_sum)
    #plot_frames(ft_err, speed)
    
def analyse_frames_from_file(file):
    df = pd.read_json(file)
    display(Markdown("# " + file))
    analyse_frames_from_dataframe(df)
    
def analyse_frames_from_runs_file(file):
    df = pd.read_json(file)
    display(Markdown("# " + file))
    by_speed_runs = []
    for index, run in df.iterrows():
        #display(Markdown("## " + json.dumps(run["params"])))
        frames = pd.DataFrame(run["frames"])
        total_error, by_speed = analyse_frames_from_dataframe(frames)
        by_speed_runs.append(pd.DataFrame({
            "size": run["params"]["tileSize"],
            "err": by_speed
        }))
        #display(total_error, by_speed)
    
    speed_err = pd.concat(by_speed_runs).sort_values(["speed", "size"])
    grouped = speed_err.groupby("speed").agg(list)
    fig, ax = plt.subplots()
    ax.set_xlabel('tile size')
    ax.set_ylabel('frame lag')
    for index, row in grouped.iterrows():
        #display(Markdown(f"## {row.name}"))
        plt.plot(row["size"], row["err"], label=f"speed {index}")
    
    leg = ax.legend();
        
    
    #display(data.iloc[0]["params"])
    #analyse_frames_from_dataframe(data)
    
#analyse_frames_from_file("scroll2.json")
#analyse_frames_from_file("scroll3-4xslow.json")
#analyse_frames_from_file("scroll4-4xslow-nosmooth-immediate.json")
#analyse_frames_from_file("scroll5-4xslow-512.json")
#analyse_frames_from_runs_file("scroll6-4xslow-256-512.json")
#analyse_frames_from_runs_file("scroll7.json")
#analyse_frames_from_runs_file("scroll8.json")
analyse_frames_from_runs_file("scroll9.json")

# scroll9.json

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.3635934630792813